### Protein Synthesis Translator
---
**Description**
>Takes DNA, RNA, codon, or amino acid sequence and returns all other corresponding sequences.
###---
**Intructions**
1. Select original sequence (DNA(template or coding)/RNA/amino acids).
2. Enter selected sequence.
3. Run

In [11]:
#@title Translator
Sequence_Type = "DNA (Template)" #@param ["DNA (Coding)", "DNA (Template)", "mRNA", "Amino Acids (Single Characters)", "Amino Acids (Multi-Character)"]
Sequence = "TAC  CGG GCT TCA TC" #@param {type:"string"}
Add_Spaces_to_Output = False #@param {type:"boolean"}
Spacing_Amount =  1#@param {type:"integer"}
Separate_Output_by_Lines = False #@param {type:"boolean"}
Quantity_Per_Line = 32 #@param {type:"integer"}
Group_Codons = True #@param {type:"boolean"}
TableTabbed_Output = False #@param {type:"boolean"}
Easy_Copy = True #@param {type:"boolean"}
Sequence = Sequence.upper().replace(' ', '').replace('\t', '')
Spacing_Amount = abs(Spacing_Amount)
Quantity_Per_Line = abs(Quantity_Per_Line)
if Quantity_Per_Line == 0:
  Quantity_Per_Line = 1
def check_sequence(sequence, sequence_type):
  valid = ['GCAT', 'GCAT', 'GCAU', 'VADEGFLSYOWCPHQRIMTNK', 'VADEGFLSYOWCPHQRIMTNK'][['DNA (Coding)', 'DNA (Template)', 'mRNA', 'Amino Acids (Single Characters)', 'Amino Acids (Multi-Character)'].index(sequence_type)]
  for character in sequence:
    if character not in valid:
      print('Invalid Character: ' + character + '\nMust be one of: ' + str(valid))
      return False
  return True
def conversion_DNA_DNA(sequence):
  product = ''
  pairings = [['G', 'C'], ['A', 'T']]
  for base in sequence:
    for pair in pairings:
      if base in pair:
        product += pair[abs(pair.index(base) - 1)]
        break
  return product
def conversion_DNA_RNA(sequence, DNA_to_RNA):
  if DNA_to_RNA:
    return conversion_DNA_DNA(sequence).replace('T', 'U')
  return conversion_DNA_DNA(sequence.replace('U', 'T'))
def conversion_RNA_AA(sequence):
  layers = [{'G':'VADEG', 'C':'lPHQR', 'A':'rsIMTNK', 'U':'oOSLFYWC'}, 
            {'G':'osGWCRr', 'C':'SAPT', 'A':'DEYHQNKO', 'U':'lLVFIM'}, 
            {'G':'ROVAEGWPQMTKLlSr', 'C':'RsVAGPTDFYCHINlS', 'A':'rRoOSlLVAGPTEQIK', 'U':'RsSlVAGPTDFYCHIN'}]
  product = ''
  codon = ''
  for base_num in range(len(sequence)):
    codon += sequence[base_num]
    if (base_num + 1) % 3 == 0:
      possible_AA = set()
      for amino_acid in layers[0].get(codon[0]):
        possible_AA.add(amino_acid)
      for next_layer_num in range(2):
        next_layer = codon[next_layer_num + 1]
        comparing_set = set()
        for amino_acid in layers[next_layer_num + 1].get(next_layer):
          comparing_set.add(amino_acid)
        possible_AA = possible_AA.intersection(comparing_set)
      for amino_acid in possible_AA:
        product += amino_acid
      codon = ''
  if codon != '':
    print('Framshift Mutation Detected: Remaining ' + codon)
  return product.upper()
def conversion_AA_AA(chain, single_to_triple):
  pairings = [['V', 'VAL'], 
              ['A', 'ALA'], 
              ['D', 'ASP'], 
              ['E', 'GLU'], 
              ['G', 'GLY'], 
              ['F', 'PHE'], 
              ['L', 'LEU'], 
              ['S', 'SER'], 
              ['Y', 'TYR'], 
              ['O', 'STO'], 
              ['W', 'TRP'], 
              ['C', 'CYS'], 
              ['P', 'PRO'], 
              ['H', 'HIS'], 
              ['Q', 'GIN'], 
              ['R', 'ARG'], 
              ['I', 'ILE'], 
              ['M', 'MET'], 
              ['T', 'THR'], 
              ['N', 'ASN'], 
              ['K', 'LYS']]
  product = ''
  if single_to_triple:
    for amino_acid in chain:
      for pair in pairings:
        if pair[0] == amino_acid:
          product += pair[1] + ' ' * Spacing_Amount
          if Spacing_Amount == 0:
            product += ' '
          break
    return product
  amino_acid = ''
  for char_num in range(len(chain)):
    amino_acid += chain[char_num]
    if (char_num + 1) % 3 == 0:
      for pair in pairings:
        if pair[1] == amino_acid:
          product += pair[0]
          break
      amino_acid = ''
  return product
def conversion_AA_RNA(chain):
  return 'Under development.'
def group_codons(sequence):
  if Group_Codons == False:
    return sequence
  sequence = sequence.replace(' ', '')
  product = ''
  crossed = 0
  codon = ''
  for base in sequence:
    crossed += 1
    codon += base
    if crossed % 3 == 0:
      product += codon + ' ' * Spacing_Amount
      codon = ''
  return product + codon
def add_newlines(sequence):
  if Separate_Output_by_Lines == False:
    return sequence
  nospaces = False
  if ' ' not in sequence:
    nospaces = True
    preproduct = []
    for character in sequence:
      preproduct.append(character)
    sequence = (' ').join(preproduct)
  product = []
  crossed = 0
  line = ''
  for index in range(len(sequence) - 1):
    character = sequence[index]
    line += character
    if character == ' ' and sequence[index + 1] != ' ':
      crossed += 1
    if crossed % Quantity_Per_Line == 0 and crossed > 0:
      product.append(line)
      line = ''
      crossed = 0
  final_product = '\n\t' + ('\n'.join(product) + '\n' + line + sequence[len(sequence) - 1]).lstrip('\n')
  if nospaces:
    final_product = final_product.replace(' ', '')
  return final_product
def add_spaces(sequence):
  if Add_Spaces_to_Output == False:
    return sequence
  product = []
  for character in sequence:
    product.append(character)
  return (' ' * Spacing_Amount).join(product)
while check_sequence(Sequence, Sequence_Type) == False:
  Sequence = input('Invalid sequence. Re-enter sequence: ').upper()
DNA_Coding = ''
DNA_Template = ''
mRNA = ''
#Notable mention to tRNA
Amino_Acids_s = ''
Amino_Acids_m = ''
if Sequence_Type == 'DNA (Coding)':
  DNA_Coding = add_spaces(Sequence)
  DNA_Template = add_spaces(conversion_DNA_DNA(Sequence))
  mRNA = add_spaces(conversion_DNA_RNA(DNA_Template.replace(' ', ''), True))
  Amino_Acids_s = add_spaces(conversion_RNA_AA(mRNA.replace(' ', '')))
  Amino_Acids_m = conversion_AA_AA(Amino_Acids_s, True)
elif Sequence_Type == 'DNA (Template)':
  DNA_Template = add_spaces(Sequence)
  DNA_Coding = add_spaces(conversion_DNA_DNA(Sequence))
  mRNA = add_spaces(conversion_DNA_RNA(DNA_Template.replace(' ', ''), True))
  Amino_Acids_s = add_spaces(conversion_RNA_AA(mRNA.replace(' ', '')))
  Amino_Acids_m = conversion_AA_AA(Amino_Acids_s, True)
elif Sequence_Type == 'mRNA':
  mRNA = add_spaces(Sequence)
  DNA_Coding = add_spaces(conversion_DNA_DNA(conversion_DNA_RNA(Sequence, False)))
  DNA_Template = conversion_DNA_RNA(Sequence, False)
  Amino_Acids_s = add_spaces(conversion_RNA_AA(mRNA.replace(' ', '')))
  Amino_Acids_m = conversion_AA_AA(Amino_Acids_s, True)
elif Sequence_Type == 'Amino Acids (Single Characters)':
  DNA_Coding = 'Under Development'
  DNA_Template = 'Under Development'
  mRNA = 'Under Development'
  Amino_Acids_s = add_spaces(Sequence)
  Amino_Acids_m = conversion_AA_AA(Sequence, True)
elif Sequence_Type == 'Amino Acids (Multi-Character)':
  DNA_Coding = 'Under Development'
  DNA_Template = 'Under Development'
  mRNA = 'Under Development'
  Amino_Acids_m = conversion_AA_AA(conversion_AA_AA(Sequence, False), True)
  Amino_Acids_s = add_spaces(conversion_AA_AA(Sequence, False))
DNA_Coding = add_newlines(group_codons(DNA_Coding))
DNA_Template = add_newlines(group_codons(DNA_Template))
mRNA = add_newlines(group_codons(mRNA))
Amino_Acids_s = add_newlines(Amino_Acids_s)
Amino_Acids_m = add_newlines(Amino_Acids_m)
if TableTabbed_Output:
  if Group_Codons == False and (Add_Spaces_to_Output == False or Spacing_Amount == 0):
    Add_Spaces_to_Output = True
    Spacing_Amount = 1
    DNA_Coding = add_spaces(DNA_Coding)
    DNA_Template = add_spaces(DNA_Template)
    mRNA = add_spaces(mRNA)
    Amino_Acids_s = add_spaces(Amino_Acids_s)
  elif Group_Codons == True and (Add_Spaces_to_Output == False or Spacing_Amount == 0):
    Add_Spaces_to_Output = True
    Spacing_Amount = 1
    Amino_Acids_s = add_spaces(Amino_Acids_s)
  DNA_Coding = DNA_Coding.replace(' ', '\t')
  DNA_Template = DNA_Template.replace(' ', '\t')
  mRNA = mRNA.replace(' ', '\t')
  Amino_Acids_s = Amino_Acids_s.replace(' ', '\t')
  Amino_Acids_m = Amino_Acids_m.replace(' ', '\t')
print('\nDNA (Coding):.. ' + DNA_Coding + '\nDNA (Template): ' + DNA_Template + '\nmRNA:.......... ' + mRNA + '\nAmino Acids (Single Characters): ' + Amino_Acids_s + '\nAmino Acids (Multi-Character):.. ' + Amino_Acids_m)
if Easy_Copy:
  print('\n' + DNA_Coding + '\n' + DNA_Template + '\n' + mRNA + '\n' + Amino_Acids_s + '\n' + Amino_Acids_m)
print('\nProgram exited.')
#fine

Framshift Mutation Detected: Remaining AG

DNA (Coding):.. ATG GCC CGA AGT AG
DNA (Template): TAC CGG GCT TCA TC
mRNA:.......... AUG GCC CGA AGU AG
Amino Acids (Single Characters): MARS
Amino Acids (Multi-Character):.. MET ALA ARG SER 

ATG GCC CGA AGT AG
TAC CGG GCT TCA TC
AUG GCC CGA AGU AG
MARS
MET ALA ARG SER 

Program exited.


###Protein Correspondance
#####Note: START and STOP are represented by M / MET and O / STO respectively. ALL refers to all bases.
- V - VAL - GUALL
- A - ALA - GCALL
- D - ASP - GAUC
- E - GLU - GAAG
- G - GLY - GGALL
- F - PHE - UUUC
- L - LEU - UUAG AND CUALL
- S - SER - UCALL AND AGCU
- Y - TYR - UAUC
- O - STO/STOP - UAAG AND UGA
- W - TRP - UGG
- C - CYS - UGUC
- P - PRO - CCALL
- H - HIS - CAUC
- Q - GIN - CAAG
- R - ARG - CGALL AND AGGA
- I - ILE - AUACU
- M - MET/START - AUG
- T - THR - ACALL
- N - ASN - AACU
- K - LYS - AAGA